In [54]:
import json
import os
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, LabelSet, CategoricalColorMapper
from bokeh.layouts import column
from bokeh.io import save

In [11]:
root = os.getcwd()

In [46]:
with open(os.path.join(root, 'jsons', 'goldStockRatiosTTM.json')) as f:
    ratios_ttm_json = json.load(f)

In [28]:
def make_comparison_scatter_plot(x_axis, y_axis):
    # x_axis and y_axis: tuple with 2 values, the first is the value in the json, the second is a pretified version
    x_list = []
    y_list = []
    names_list = []
    x_tickers = 0
    y_tickers = 0
    both_tickers = 0
    for ticker, data in ratios_ttm_json.items():
        data = data[0]
        if data[x_axis[0]]:
            x_tickers += 1
        if data[y_axis[0]]:
            y_tickers += 1
        if data[x_axis[0]] and data[y_axis[0]]:
            both_tickers += 1
            x_list.append(data[x_axis[0]])
            y_list.append(data[y_axis[0]])
            names_list.append(ticker)

    print(f"Companies with {x_axis[1]}: {x_tickers}")
    print(f"Companies with {y_axis[1]}: {y_tickers}")
    print(f"Companies with both {x_axis[1]} and {y_axis[1]}: {both_tickers}")
    source = ColumnDataSource(data={f"{x_axis[1]}": x_list,
                                    f"{y_axis[1]}": y_list,
                                    "Tickers": names_list})
    p = figure(title=f"{y_axis[1]} TTM vs {x_axis[1]} TTM of Prominent Gold Stocks")
    p.scatter(x=x_axis[1], y=y_axis[1], source=source)
    p.xaxis.axis_label = x_axis[1]
    p.yaxis.axis_label = y_axis[1]
    labels = LabelSet(x=x_axis[1], y=y_axis[1], text="Tickers", source=source, x_offset=5, y_offset=5)
    p.add_layout(labels)
    show(p)
make_comparison_scatter_plot(("quickRatioTTM", "Quick Ratio"), ('operatingProfitMarginTTM', "Operating Profit Margin"))

Companies with Quick Ratio: 54
Companies with Operating Profit Margin: 43
Companies with both Quick Ratio and Operating Profit Margin: 42


## Finding how many of each stats are available

In [27]:
ratios_dict = {}
for ticker, data in ratios_ttm_json.items():
    data = data[0]
    for key, item in data.items():
        if key not in ratios_dict:
            ratios_dict[key] = 0
        if item:
            ratios_dict[key] += 1
ratios_dict

{'dividendYielTTM': 20,
 'dividendYielPercentageTTM': 20,
 'peRatioTTM': 18,
 'pegRatioTTM': 18,
 'payoutRatioTTM': 18,
 'currentRatioTTM': 54,
 'quickRatioTTM': 54,
 'cashRatioTTM': 54,
 'daysOfSalesOutstandingTTM': 40,
 'daysOfInventoryOutstandingTTM': 41,
 'operatingCycleTTM': 42,
 'daysOfPayablesOutstandingTTM': 46,
 'cashConversionCycleTTM': 42,
 'grossProfitMarginTTM': 43,
 'operatingProfitMarginTTM': 43,
 'pretaxProfitMarginTTM': 43,
 'netProfitMarginTTM': 43,
 'effectiveTaxRateTTM': 48,
 'returnOnAssetsTTM': 54,
 'returnOnEquityTTM': 54,
 'returnOnCapitalEmployedTTM': 54,
 'netIncomePerEBTTTM': 55,
 'ebtPerEbitTTM': 55,
 'ebitPerRevenueTTM': 43,
 'debtRatioTTM': 54,
 'debtEquityRatioTTM': 54,
 'longTermDebtToCapitalizationTTM': 36,
 'totalDebtToCapitalizationTTM': 16,
 'interestCoverageTTM': 51,
 'cashFlowToDebtRatioTTM': 16,
 'companyEquityMultiplierTTM': 54,
 'receivablesTurnoverTTM': 40,
 'payablesTurnoverTTM': 46,
 'inventoryTurnoverTTM': 41,
 'fixedAssetTurnoverTTM': 41,
 

In [56]:
width = 1700
height = 800
def make_bar_charts_for_ratios(ratios_list):
    """
    ratios_list is a list of tuples, the first one is the ratio name in the json, the second one is the clean name, the third one is true if a higher number is 'better'
    """
    figures_list = []
    for ratio in ratios_list:
        ratio_values_list = []
        ticker_list = []
        for ticker, data in ratios_ttm_json.items():
            data = data[0]
            ratio_value = data[ratio[0]]
            if ratio_value:
                ratio_values_list.append(ratio_value)
                ticker_list.append(ticker)
        sorted_tickers_list = sorted(ticker_list, key=lambda x: -ratio_values_list[ticker_list.index(x)])
        if not ratio[2]:
            sorted_tickers_list.reverse()
        # Only take the best 20 stocks for each ratio
        end_index = 20 if len(sorted_tickers_list) >= 20 else len(sorted_tickers_list)
        sorted_tickers_list = sorted_tickers_list[0:end_index]
        accepted_indexes = []
        for ind, val in enumerate(ticker_list):
            if val in sorted_tickers_list:
                accepted_indexes.append(ind)
        ticker_list = [ticker_list[i] for i in accepted_indexes]
        ratio_values_list = [ratio_values_list[i] for i in accepted_indexes]
        color_mapper = CategoricalColorMapper(factors=['KGC', 'TXG.TO', 'CEE.TO'], palette=["red", "green", "orange"], nan_color="blue")
        p = figure(width=width, height=height, x_range=sorted_tickers_list, title=f"{ratio[1]} TTM for Gold Stocks")
        source = ColumnDataSource({"Ticker": ticker_list,
                                   ratio[1]: ratio_values_list})
        p.vbar(x="Ticker", top=ratio[1], width=0.9, fill_color={'field': 'Ticker', 'transform': color_mapper}, source=source)
        figures_list.append(p)
#     show(column(figures_list))
    return column(figures_list)
    
    
f = make_bar_charts_for_ratios([("quickRatioTTM", "Quick Ratio", True), 
                                    ('operatingProfitMarginTTM', "Operating Profit Margin", True), 
                                    ('returnOnEquityTTM', "Return On Equity", True),
                                    ('peRatioTTM', "Price to Earnings", False)])
save(f, os.path.join(root, 'plots', "ttm_ratio_bar_charts.html"))

            

'/home/ben_luft/JupyterProjects/goldStocks/plots/ttm_ratio_bar_charts.html'

In [57]:
print(len(ratios_ttm_json))

66
